In [1]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt

from batchAQUA_GPU import batchAQUA_GPU

In [2]:
T = 10
dt = 0.01
N_iter = int(T/dt)

N_neurons = 5000

x_ini = np.array([0, 0, 0])
x_start = np.full((N_neurons, 3), fill_value = x_ini)
t_start = np.zeros(N_neurons)

RS = {'name': 'RS', 'C': 100, 'k': 0.7, 'v_r': -60, 'v_t': -40, 'v_peak': 35,
     'a': 0.03, 'b': -2, 'c': -50, 'd': 100}    # Class 1

params = []
for i in range(N_neurons):
    params.append(RS)

batch = batchAQUA_GPU(params)
batch.Initialise(x_start, t_start)

I_inj = cp.array([i*np.ones(T) for i in range(N_neurons)])




CUDARuntimeError: cudaErrorInsufficientDriver: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
# simulate
X, t, spikes = batch.update_RK2(dt, N_iter, I_inj)